In [1]:
from itertools import combinations

def generate_candidates(itemsets, length):

    return set([item1.union(item2) for item1 in itemsets for item2 in itemsets if len(item1.union(item2)) == length])


def filter_candidates(transactions, candidates, min_support):

    itemset_counts = {candidate: 0 for candidate in candidates}
    for transaction in transactions:
        for candidate in candidates:
            if candidate.issubset(transaction):
                itemset_counts[candidate] += 1
    return {itemset: count for itemset, count in itemset_counts.items() if count >= min_support}

In [2]:

def apriori(transactions, min_support):
    # Generate 1-itemsets
    itemsets = set(frozenset([item]) for transaction in transactions for item in transaction)
    frequent_itemsets = filter_candidates(transactions, itemsets, min_support)
    result = dict(frequent_itemsets)

    k = 2
    while frequent_itemsets:
        # Generate candidates of size k
        candidates = generate_candidates(frequent_itemsets.keys(), k)
        # Filter candidates
        frequent_itemsets = filter_candidates(transactions, candidates, min_support)
        result.update(frequent_itemsets)
        k += 1
    return result


def association_rules(frequent_itemsets, min_confidence):

    rules = []
    for itemset in frequent_itemsets:
        for length in range(1, len(itemset)):
            for subset in combinations(itemset, length):
                antecedent = frozenset(subset)
                consequent = itemset - antecedent
                support = frequent_itemsets[itemset]
                confidence = support / frequent_itemsets[antecedent] if frequent_itemsets.get(antecedent) else 0
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence))
    return rules



In [3]:

transactions = [
    {'milk', 'bread', 'butter'},
    {'beer', 'bread'},
    {'milk', 'bread', 'beer', 'butter'},
    {'beer', 'butter'},
    {'bread', 'butter'}
]
min_support = 2
min_confidence = 0.6

frequent_itemsets = apriori(transactions, min_support)
rules = association_rules(frequent_itemsets, min_confidence)

print("Frequent Itemsets:", frequent_itemsets)
print("Association Rules:")
for antecedent, consequent, confidence in rules:
    print(f"{set(antecedent)} -> {set(consequent)} (confidence: {confidence:.2f})")

Frequent Itemsets: {frozenset({'milk'}): 2, frozenset({'butter'}): 4, frozenset({'beer'}): 3, frozenset({'bread'}): 4, frozenset({'milk', 'butter'}): 2, frozenset({'butter', 'bread'}): 3, frozenset({'milk', 'bread'}): 2, frozenset({'beer', 'butter'}): 2, frozenset({'beer', 'bread'}): 2, frozenset({'milk', 'butter', 'bread'}): 2}
Association Rules:
{'milk'} -> {'butter'} (confidence: 1.00)
{'butter'} -> {'bread'} (confidence: 0.75)
{'bread'} -> {'butter'} (confidence: 0.75)
{'milk'} -> {'bread'} (confidence: 1.00)
{'beer'} -> {'butter'} (confidence: 0.67)
{'beer'} -> {'bread'} (confidence: 0.67)
{'milk'} -> {'butter', 'bread'} (confidence: 1.00)
{'milk', 'butter'} -> {'bread'} (confidence: 1.00)
{'milk', 'bread'} -> {'butter'} (confidence: 1.00)
{'butter', 'bread'} -> {'milk'} (confidence: 0.67)
